## SECP3223-02 Project - WHO TB Infection Estimates in Household Contacts

## Group DataPioneers

### Muhammad Nur Azhar Bin Mohd Yazid (A22EC0220) - Project Lead
### Che Marhumi Bin Che Ab Rahim (A22EC0147)
### Muhammad Safwan Bin Mohd Azmi (A22EC0221)

## Data Dictionary

| **Column Name**                 | **Description**                                                                                     | **Data Type** |
|----------------------------------|-----------------------------------------------------------------------------------------------------|---------------|
| `country`                       | Name of the country                                                                                 | Category      |
| `iso2`                          | 2-letter ISO country code (some missing values)                                                     | Category      |
| `iso3`                          | 3-letter ISO country code                                                                            | Category      |
| `iso_numeric`                   | Numeric ISO country code                                                                             | Integer       |
| `g_whoregion`                   | WHO regional affiliation                                                                             | Category      |
| `year`                          | Year of the estimate                                                                                 | Integer       |
| `source_hh`                     | Source of household size data                                                                        | Category      |
| `e_hh_size`                     | Estimated average household size                                                                     | Float         |
| `prevtx_data_available`         | Indicator of whether previous treatment data is available                                            | Float         |
| `newinc_con_prevtx`             | New TB cases in contacts with previous treatment (partially available)                              | Float         |
| `newinc_con04_prevtx`           | New TB cases in children under 5 years in contacts with previous treatment                           | Float         |
| `e_hh_contacts`                 | Estimated total household contacts                                                                   | Float         |
| `e_hh_contacts_lo`              | Lower bound of estimated household contacts                                                         | Float         |
| `e_hh_contacts_hi`              | Upper bound of estimated household contacts                                                         | Float         |
| `e_prevtx_hh_contacts_pct`      | Percentage of household contacts with previous TB treatment                                          | Float         |
| `e_prevtx_hh_contacts_pct_lo`   | Lower bound of percentage of household contacts with previous TB treatment                          | Float         |
| `e_prevtx_hh_contacts_pct_hi`   | Upper bound of percentage of household contacts with previous TB treatment                          | Float         |
| `e_prevtx_eligible`             | Estimated number of individuals eligible for treatment                                               | Float         |
| `e_prevtx_eligible_lo`          | Lower bound of estimated individuals eligible for treatment                                          | Float         |
| `e_prevtx_eligible_hi`          | Upper bound of estimated individuals eligible for treatment                                          | Float         |
| `e_prevtx_kids_pct`             | Percentage of eligible individuals who are children                                                  | Float         |
| `e_prevtx_kids_pct_lo`          | Lower bound of the percentage of eligible children                                                   | Float         |
| `e_prevtx_kids_pct_hi`          | Upper bound of the percentage of eligible children                                                   | Float         |
| `household_contact_rate`        | Derived feature: household contact rate (total contacts divided by household size)                   | Float         |


## Part A: Data Cleaning and Preparation
### Step 1: Load the dataset

In [2]:
import pandas as pd

file_path = 'LTBI_estimates.csv'
tb_data = pd.read_csv(file_path)

### Step 2: Understanding the Data

In [3]:
# Basic info and summary statistics
tb_data.info()
tb_data.describe(include='all')

# Check for missing values
missing_summary = tb_data.isnull().sum()
print(missing_summary)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1710 entries, 0 to 1709
Data columns (total 25 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   country                       1710 non-null   object 
 1   iso2                          1701 non-null   object 
 2   iso3                          1710 non-null   object 
 3   iso_numeric                   1710 non-null   int64  
 4   g_whoregion                   1710 non-null   object 
 5   year                          1710 non-null   int64  
 6   source_hh                     1710 non-null   object 
 7   e_hh_size                     1710 non-null   float64
 8   prevtx_data_available         1564 non-null   float64
 9   newinc_con_prevtx             753 non-null    float64
 10  newinc_con04_prevtx           1068 non-null   float64
 11  ptsurvey_newinc               14 non-null     float64
 12  ptsurvey_newinc_con04_prevtx  14 non-null     float64
 13  e_h

### Step 3: Handling Missing Values

In [4]:
# (a) Drop columns with excessive missing values (>90% missing)
columns_to_drop = ['ptsurvey_newinc', 'ptsurvey_newinc_con04_prevtx']
tb_data_cleaned = tb_data.drop(columns=columns_to_drop)

# (b) Fill numeric missing values with the median
numeric_cols = tb_data_cleaned.select_dtypes(include=['float64', 'int64']).columns
for col in numeric_cols:
    if tb_data_cleaned[col].isnull().sum() > 0:
        tb_data_cleaned[col].fillna(tb_data_cleaned[col].median(), inplace=True)

# (c) Fill categorical missing values with the mode
categorical_cols = tb_data_cleaned.select_dtypes(include=['object', 'category']).columns
for col in categorical_cols:
    if tb_data_cleaned[col].isnull().sum() > 0:
        tb_data_cleaned[col].fillna(tb_data_cleaned[col].mode()[0], inplace=True)

C:\Users\MNAZHAR\AppData\Local\Temp\ipykernel_44760\3112744729.py:9: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  tb_data_cleaned[col].fillna(tb_data_cleaned[col].median(), inplace=True)
C:\Users\MNAZHAR\AppData\Local\Temp\ipykernel_44760\3112744729.py:15: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values alway

### Step 4: Standardize Data Types

In [5]:
# Convert categorical columns to the 'category' type
tb_data_cleaned['country'] = tb_data_cleaned['country'].astype('category')
tb_data_cleaned['iso3'] = tb_data_cleaned['iso3'].astype('category')
tb_data_cleaned['g_whoregion'] = tb_data_cleaned['g_whoregion'].astype('category')
tb_data_cleaned['source_hh'] = tb_data_cleaned['source_hh'].astype('category')


### Step 5: Derive New Features

In [6]:
# Calculate household_contact_rate
tb_data_cleaned['household_contact_rate'] = tb_data_cleaned['e_hh_contacts'] / tb_data_cleaned['e_hh_size']

### Step 6: Handle Inconsistent or Duplicate Rows

In [7]:
tb_data_cleaned.dropna(subset=['country', 'iso3', 'year'], inplace=True)

tb_data_cleaned = tb_data_cleaned.drop_duplicates()

### Step 7: Validate Data Integrity

In [8]:
assert (tb_data_cleaned['e_prevtx_eligible_lo'] <= tb_data_cleaned['e_prevtx_eligible']).all()
assert (tb_data_cleaned['e_prevtx_eligible'] <= tb_data_cleaned['e_prevtx_eligible_hi']).all()

print(tb_data_cleaned.isnull().sum())


country                        0
iso2                           0
iso3                           0
iso_numeric                    0
g_whoregion                    0
year                           0
source_hh                      0
e_hh_size                      0
prevtx_data_available          0
newinc_con_prevtx              0
newinc_con04_prevtx            0
e_hh_contacts                  0
e_hh_contacts_lo               0
e_hh_contacts_hi               0
e_prevtx_hh_contacts_pct       0
e_prevtx_hh_contacts_pct_lo    0
e_prevtx_hh_contacts_pct_hi    0
e_prevtx_eligible              0
e_prevtx_eligible_lo           0
e_prevtx_eligible_hi           0
e_prevtx_kids_pct              0
e_prevtx_kids_pct_lo           0
e_prevtx_kids_pct_hi           0
household_contact_rate         0
dtype: int64


### Step 8: Export Cleaned Data

In [9]:
# Save the cleaned dataset for further analysis
tb_data_cleaned.to_csv('LTBI_cleaned_data.csv', index=False)